# Nested workflows
This notebook demos a nested workflow approach. The parent defines a process of two steps, each of which is a sub workflow. 

In the parent transitions are defined to move from one step to the next - thus they are sequential.
Children workflows instead are free to move between agents.

## Context
A fake Contoso Insurance company is used to demonstrate the process. The company has a process of handling refunds. The process is divided into two steps:

## Process

1. Parent workflow
    1. Team `Step1`: Data collection
        1. Agent `data_collector` collects data from the provided ticket
        2. Agent `data_validator` validates the data and potentially resends the ticket to the `data_collector`
    2. Team `Step2`: Refund handling
        1. Agent `approver` processes the refund if amount is less than 1000 USD, otherwise sends it to the `approval_manager`
        2. User `approval_manager` approves the refund if amount is more than 1000 USD

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.agent import Agent
from vanilla_aiagents.user import User
from vanilla_aiagents.workflow import Workflow
from vanilla_aiagents.team import Team
from vanilla_aiagents.llm import AzureOpenAILLM
from dotenv import load_dotenv

load_dotenv(override=True)
import os

In [4]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.user").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)

In [ ]:
data_collector = Agent(id="data_collector", description="Collects information from the provided input", llm=llm, system_message="""
You are part of an automated process that extracts data from a user-provided ticket.
Your task is to extract the following information from the ticket:
- User name and email
- User's request
- User's inquiried amount of money
- Accident date (in ISO format)
- Accident location
- Accident description
- Accident kind
- Insurance company
- Insurance policy number

When the accident kind regards the following options, look for additional information:
- Car accident:
    - Car brand and model
    - Car license plate
    - Other car's license plate
    - Other car's brand and model
- Home accident
    - Home address
    - Room where the accident happened
""")

In [ ]:
data_evaluator = Agent(id="data_evaluator", description="Evaluates the data collected by the data collector", llm=llm, system_message="""
You are part of an automated process that evaluates the data collected by the data collector.
Your task is to evaluate the following information extracted from the ticket:
- User name and email
- User's request
- Accident date (in ISO format)
- Accident location
- Accident description
- Accident kind
- Insurance company
- Insurance policy number

When the accident kind regards the following options, look for additional information:
- Car accident:
    - Car brand and model
    - Car license plate
    - Other car's license plate
    - Other car's brand and model
- Home accident
    - Home address
    - Room where the accident happened
    
    
OUTPUT:
- 'DATA COLLECTION DONE' if the data is correct
- 'REPEAT DATA COLLECTION' if the data is incorrect
""")

In [ ]:
step1 = Team(id="step1", description="Validates the input provided to ensure all required information are collected", 
             members=[data_collector, data_evaluator], 
             system_prompt="",
             llm=llm, 
             stop_callback=lambda conv: conv.messages[-1]['content'].lower().endswith("done"))

In [ ]:
approver = Agent(id="approver", description="Approves the data collected by the data collector", llm=llm, system_message="""
You are part of an automated process that needs to approve or forward money refunds to a user insurance claim.

If the amount is less than 1000 USD, you can approve the refund. Otherwise, you need to forward the request to the manager.

OUTPUT:
- 'APPROVE' if the amount is less than 1000 USD
- 'FORWARD' if the amount is greater than 1000 USD
""")

In [9]:
approval_manager = User(id="approval_manager", description="An insurance manager that can approve or reject the refund request when the automatic approver forwards the request")

In [ ]:
step2 = Team(id="step2", description="Processes the information gathered to provide the final answer", members=[approver, approval_manager], llm=llm, 
    stop_callback=lambda conv: conv.messages[-1]['content'].lower().endswith("approve") or conv.messages[-1]['content'].lower().endswith("reject"))

In [11]:
from vanilla_aiagents.sequence import Sequence
process = Sequence(id="process", description="The process team", steps=[step1, step2], llm=llm)

In [12]:
ticket= """
From: Alice Thompson <a_thompson@foo.com>
To: Contoso Insurance Team <report@contoso-insurancec.com>
Subject: URGENT: Car Accident Claim - Immediate Assistance Required

Dear Contoso Insurance Team,

I hope this email finds you well. I am writing to you with a heavy heart and a great deal of stress regarding an unfortunate incident that occurred earlier today. I was involved in a car accident and I am seeking immediate assistance to file a claim and get the support I need during this challenging time.

Here's what happened: I was driving home from work, heading south on Maple Avenue, when out of nowhere, another vehicle came speeding through a red light at the intersection of Maple and Oak Street. I had no time to react, and the other car crashed into the passenger side of my vehicle with considerable force. The impact was severe, and both cars were significantly damaged. My car, a 2018 Toyota Camry, has extensive damage to the passenger side, and I believe it may not be drivable at this point.

Thankfully, I did not sustain any major injuries, but I am feeling quite shaken up and have some minor bruises. The other driver appeared to be unharmed as well, but their car, a silver Honda Civic, was also badly damaged. We exchanged contact and insurance information at the scene, and I made sure to take photos of the damage to both vehicles for documentation purposes.

The accident was promptly reported to the local authorities, and a police report was filed. I have attached a copy of the police report, along with the photos I took, for your reference. Additionally, I have provided my insurance policy number and other relevant details below to expedite the process.

Policy Number: 2021-123456789
Date of Accident: Monday, August 23, 2022
Location: Intersection of Maple Avenue and Oak Street
Vehicle Involved: 2018 Toyota Camry (License Plate: AAA-1234)
Other Party Involved: Silver Honda Civic (License Plate: ZZZ-5678)

I am deeply concerned about the repair costs ($ 1000) and the potential need for a rental car while my vehicle is being repaired. I would greatly appreciate it if you could guide me through the next steps and let me know what information or documentation you require from my end to process the claim efficiently.

This is a very distressing situation for me, and I am relying on your prompt assistance and expertise to help me navigate this process. Please let me know if there are any forms I need to fill out or additional information I need to provide.

Thank you in advance for your understanding and support during this difficult time. I look forward to hearing from you soon and hope for a swift resolution to my claim.

Warm regards,

Alice Thompson
"""

In [ ]:
workflow = Workflow(process)
workflow.restart()
# workflow.run(ticket)
stream = []
for mark, content in workflow.run_stream(ticket):
    print(f"{mark}: {content}")
    stream.append({"mark": mark, "content": content})

In [ ]:
workflow.conversation.messages

In [ ]:
# print stream element by element, but only when mark is start, end or response
for i in range(len(stream)):
    if stream[i]["mark"] in ["start", "end", "response"]:
        print(f"{stream[i]['mark']}: {stream[i]['content']}")